<a href="https://colab.research.google.com/github/SaeidDaliriSusefi/SoilMoisture-S2/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt

In [3]:
ee.Authenticate()
ee.Initialize(project="ee-saeiddalirisu", opt_url='https://earthengine-highvolume.googleapis.com')

In [70]:
# Define variables
country_name = 'Italy'
region_name = 'Lombardia'
city_name = 'Lecco'
time_start = '2024-01-01'
time_end = '2024-12-30'

In [71]:
# Load and filter administrative boundaries
roi = (ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
          .filter(ee.Filter.eq('ADM0_NAME', country_name))
          .filter(ee.Filter.eq('ADM1_NAME', region_name))
          .filter(ee.Filter.eq('ADM2_NAME', city_name)))

In [72]:
sentinel = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(roi)
    .filterDate(time_start, time_end)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 45))
)


In [73]:
parameters = sentinel.map(lambda img:
    (ee.Image.cat([
        img.select('B.*').multiply(0.0001).normalizedDifference(['B8', 'B4']).rename('ndvi'),
        img.select('B.*').multiply(0.0001)
           .expression('((1 - swir) ** 2) / (2 * swir)', {'swir': img.select('B.*').multiply(0.0001).select('B12')})
           .rename('str')
    ])
    .copyProperties(img, img.propertyNames()))
)

In [74]:
str_full_cover = parameters.map(lambda img:
    (img.select('str')
        .updateMask(img.select('ndvi').gt(0.3))
        .copyProperties(img, img.propertyNames()))
)

In [75]:
str_bareland = parameters.map(lambda img:
    (img.select('str')
        .updateMask(
            img.select('ndvi').gte(0).And(img.select('ndvi').lt(0.2))
        )
        .copyProperties(img, img.propertyNames()))
)

In [76]:
vw = ee.Number(str_full_cover.max().reduceRegion(
    reducer=ee.Reducer.max(), geometry=roi, scale=100
).values().get(0))

vd = ee.Number(str_full_cover.min().reduceRegion(
    reducer=ee.Reducer.min(), geometry=roi, scale=100
).values().get(0))

iw = ee.Number(str_bareland.max().reduceRegion(
    reducer=ee.Reducer.max(), geometry=roi, scale=100
).values().get(0))

id = ee.Number(str_bareland.min().reduceRegion(
    reducer=ee.Reducer.min(), geometry=roi, scale=100
).values().get(0))

sw = vw.subtract(iw)
sd = vd.subtract(id)

In [77]:
sm = parameters.map(lambda img:
    (img.expression(
        '(id + sd * ndvi - str) / (id - iw + (sd - sw) * ndvi)', {
            'id': id,
            'sd': sd,
            'ndvi': img.select('ndvi'),
            'str': img.select('str'),
            'iw': iw,
            'sw': sw
        })
    .rename('soil_moisture')
    .multiply(1000)
    .updateMask(img.select('ndvi').lt(-0.1).Not())
    .copyProperties(img, img.propertyNames()))
)

In [78]:
def get_band_values(img):
    image_date = img.get('system:time_start')
    image_date = ee.Date(image_date).format('YYYY-MM-dd').getInfo()

    region_values = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=30,
        maxPixels=1e8
    )

    return (image_date, region_values)

band_values_dict = {}

for i in range(sm.size().getInfo()):
    img = sm.toList(sm.size()).get(i)
    date, band_values = get_band_values(ee.Image(img))
    band_values_dict[date] = band_values.getInfo()  # Add the data to the dictionary

import matplotlib.pyplot as plt


dates = []
soil_moisture_values = []

for date, band_values in band_values_dict.items():
    if 'soil_moisture' in band_values:  # Check if the soil moisture band exists
        soil_moisture_values.append(band_values['soil_moisture'])
        dates.append(date)

date_nums = np.arange(len(dates))

slope, intercept = np.polyfit(date_nums, soil_moisture_values, 1)
trendline = np.polyval([slope, intercept], date_nums)

plt.figure(figsize=(20, 6))
plt.plot(dates, soil_moisture_values, marker='o', linestyle='-', color='b', label='Soil Moisture')
plt.plot(dates, trendline, color='r', linestyle='--', label='Trendline')


plt.xlabel('Date')
plt.ylabel('Soil Moisture')
plt.title('Soil Moisture over Time')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('soil_moisture_trendline.png', format='png')

KeyboardInterrupt: 